In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import find_peaks, detrend, butter, filtfilt
from sklearn.metrics import roc_auc_score, roc_curve
import math
from scipy.fft import rfftfreq, rfft # AUC計算コードには不要だが、元のコードのimportを維持

# ========= 基本設定 =========
fs = 2000  # Hz
depths = [3, 9, 15, 21]
state_map = {1: "Normal", 2: "Ischaemia", 3: "Congestion"}
channels = {"ppgA_Red_AGC": "Red", "ppgA_IR_AGC": "IR"} 
DEBUG_MODE = True
DEBUG_FILE = "debug_log_part1_to_3_time_features_norm.txt"
with open(DEBUG_FILE, "w") as f:
    f.write("--- Debug Log Start (Time Features BPM Norm) ---\n")

def DEBUG_LOG(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a") as f:
            f.write(message + "\n")
        # print(message)

# --- BPF, BPM推定関数 (変更なし) ---
def bandpass_filter(sig, fs, low=0.5, high=5.0, order=3):
    nyq = fs / 2
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    return filtfilt(b, a, sig)

def estimate_bpm(signal, fs=2000):
    sig = pd.Series(signal, dtype=float).interpolate().fillna(0).values
    sig = detrend(sig)
    sig = bandpass_filter(sig, fs)
    prom = max(np.std(sig) * 0.3, np.ptp(sig) * 0.05)
    peaks, _ = find_peaks(sig, prominence=prom, distance=int(fs * 0.5))
    if len(peaks) < 2:
        return np.nan
    duration_sec = len(sig) / fs
    bpm = len(peaks) / (duration_sec / 60.0)
    return bpm

# --- 1拍ごとの特徴抽出 (ロバスト性向上) ---
def extract_pulse_features(signal, fs, dist=0.3):
    threshold_height = np.min(signal) + (np.ptp(signal) * 0.1)
    peaks, _ = find_peaks(signal, height=threshold_height, distance=int(fs * dist))
    
    feats = []
    if len(peaks) < 2: return []
        
    for p in peaks:
        left = max(0, p - int(0.5 * fs))
        right = min(len(signal), p + int(0.5 * fs))
        seg = signal[left:right]
        if len(seg) < 5: continue
        if np.any(np.isnan(seg)) or np.all(seg == seg[0]): continue
        
        peak_idx = np.argmax(seg)
        trough_idx = np.argmin(seg[:peak_idx+1])
        next_trough_idx = peak_idx + np.argmin(seg[peak_idx:])
        
        if trough_idx < peak_idx < next_trough_idx:
            
            rise = (peak_idx - trough_idx) / fs
            fall = (next_trough_idx - peak_idx) / fs
            baseline = seg[trough_idx]
            peak_val = seg[peak_idx]
            amp = peak_val - baseline
            half = baseline + 0.5 * amp
            
            # Pulse Width (PW)
            seg_pw = seg[trough_idx:next_trough_idx+1]
            above_half = np.where(seg_pw >= half)[0]
            pw = (above_half[-1] - above_half[0]) / fs if len(above_half) > 1 else np.nan
            
            # Systolic Width (SW)
            systolic_seg = seg[trough_idx:peak_idx+1]
            sys_above_half = np.where(systolic_seg >= half)[0]
            sw = (sys_above_half[-1] - sys_above_half[0]) / fs if len(sys_above_half) > 1 else np.nan
            
            # Diastolic Width (DW)
            diastolic_seg = seg[peak_idx:next_trough_idx+1]
            dias_above_half = np.where(diastolic_seg >= half)[0]
            dw = (dias_above_half[-1] - dias_above_half[0]) / fs if len(dias_above_half) > 1 else np.nan
            
            feats.append((rise, fall, pw, amp, sw, dw))
    return feats

# ========= Part 1 & 2: データロードと特徴抽出 =========
records = []
pulse_id_counter = 0 

for depth in depths:
    for i_state, state_name in state_map.items():
        file_path = Path(f"{depth}mm_AGC_data{i_state}.csv")
        DEBUG_LOG(f"\n--- Processing: {file_path.name} ({state_name}) ---")

        if not file_path.exists(): continue
        df = pd.read_csv(file_path)

        # --- BPM推定 ---
        ir_sig = df.get("ppgA_IR_AGC", pd.Series([]).values)
        if len(ir_sig) == 0: continue
            
        bpm = estimate_bpm(ir_sig, fs)
        window_sec = 60.0 / bpm if not np.isnan(bpm) and bpm > 0 else 1.0
        window_sec = float(np.clip(window_sec, 0.6, 2.0))
        
        step = int(round(fs * window_sec))
        current_pulse_id = pulse_id_counter 

        for col, ch_label in channels.items():
            if col not in df.columns: continue
            sig = df[col].values

            # (1) Window-based Amplitude 
            n_win = len(sig) // step
            DEBUG_LOG(f"  Channel: {ch_label}, Total Windows: {n_win}")
            
            for w in range(n_win):
                s, e = w * step, (w + 1) * step
                seg = sig[s:e]
                if len(seg) < 5: continue
                amp = seg.max() - seg.min()
                
                # ★修正: Time featuresのみに限定するため、Amplitudeはここでは記録しない
                pass

            # (2) peaksで詳細特徴抽出 (Peak-based)
            feats = extract_pulse_features(sig, fs)
            DEBUG_LOG(f"  Channel: {ch_label}, Extracted Pulse Features: {len(feats)} pulses.")
            
            for rise, fall, pw, amp, sw, dw in feats:
                # ★★★ BPM値とPulse Intervalを格納 ★★★
                records.append({
                    "Depth": depth, "State": state_name, "Channel": ch_label,
                    "Pulse_ID": current_pulse_id,
                    "RiseTime": rise, 
                    "FallTime": fall,
                    "PulseWidth": pw, 
                    "Amplitude": amp,
                    "SystolicWidth": sw, 
                    "DiastolicWidth": dw, 
                    "BPM": bpm,
                    "PulseInterval": window_sec # Pulse Interval = Window Sec
                })
        
        pulse_id_counter += 1 

df_feat = pd.DataFrame(records)
if df_feat.empty:
    raise RuntimeError("Error: No data loaded. Please check CSV files.")


# ========= Part 3: 複合指標と正規化 (時間特徴に特化) =========

# ★★★ 修正: BPM正規化された時間的特徴量の計算 ★★★
time_features = ["RiseTime", "FallTime", "PulseWidth", "SystolicWidth", "DiastolicWidth"]

for f in time_features:
    # PulseIntervalで除算し、BPM依存性を除去
    df_feat[f"{f}_norm_bpm"] = df_feat[f] / df_feat["PulseInterval"]

# 複合指標の計算 (BPM正規化後の特徴量を使用)
df_feat["RiseFall_ratio"] = df_feat["FallTime"] / (df_feat["RiseTime"] + 1e-6) # BPM正規化は不要 (比率のため)
df_feat["Width_Ratio"] = df_feat["DiastolicWidth"] / (df_feat["SystolicWidth"] + 1e-6) # BPM正規化は不要 (比率のため)

DEBUG_LOG(f"\n--- Part 3 Complete: BPM Normalized Time Features Created ---")


# --- Red/IR比 (マージロジックのみ使用) ---
amp_data = df_feat.dropna(subset=['Amplitude']).copy()

# マージに必要なカラムリスト (正規化後の時間特徴量を含む)
merge_cols = ['Depth', 'State', 'Pulse_ID'] + time_features + [f"{f}_norm_bpm" for f in time_features] + ["RiseFall_ratio", "Width_Ratio"] + ['Amplitude']


df_red = amp_data[amp_data.Channel == "Red"][merge_cols].rename(
    columns={"Amplitude": "Amplitude_Red"}
)

df_ir = amp_data[amp_data.Channel == "IR"][merge_cols].rename(
    columns={"Amplitude": "Amplitude_IR"}
)

df_ratio = pd.merge(df_red, df_ir, on=['Depth','State','Pulse_ID'], how='inner', suffixes=('_Red', '_IR'))
DEBUG_LOG(f"Ratio Data Rows (Merged for Time Features): {len(df_ratio)}")


# ========= Part 4: ROC AUC計算 (時間特徴に限定) =========
pairs = [("Normal","Ischaemia"),("Normal","Congestion"),("Ischaemia","Congestion")]

# ★★★ 評価対象の特徴量リストを更新 ★★★
features_to_eval = [
    "RiseTime", "FallTime", "PulseWidth", "SystolicWidth", "DiastolicWidth",
    "RiseTime_norm_bpm", "FallTime_norm_bpm", "PulseWidth_norm_bpm", 
    "SystolicWidth_norm_bpm", "DiastolicWidth_norm_bpm",
    "RiseFall_ratio", "Width_Ratio"
] 

# AUC Bootstrap Helper (再定義)
def bootstrap_auc_ci(y_true, y_score, n_bootstrap=200):
    rng = np.random.default_rng(42)
    aucs=[]
    n=len(y_true)
    for _ in range(n_bootstrap):
        idx = rng.integers(0,n,n)
        if len(np.unique(y_true[idx]))<2 or np.all(y_score[idx]==y_score[idx][0]): continue
        aucs.append(roc_auc_score(y_true[idx],y_score[idx]))
    if not aucs: return np.nan,np.nan,np.nan
    return np.mean(aucs), np.percentile(aucs,2.5), np.percentile(aucs,97.5)


rows=[]
# Channel-specific features
for ch in ["Red","IR"]:
    for depth in depths:
        sub = df_feat[(df_feat.Channel==ch)&(df_feat.Depth==depth)]
        if sub.empty: continue
        for f in features_to_eval:
            for c1,c2 in pairs:
                d=sub[sub.State.isin([c1,c2])]
                if d.empty: continue
                y_true=(d.State==c1).astype(int).values
                y_score=d[f].values
                mask=~np.isnan(y_score)
                y_true=y_true[mask]; y_score=y_score[mask]
                
                n_samples_used = len(y_true)
                if n_samples_used < 5 or len(np.unique(y_true))<2: 
                    DEBUG_LOG(f"[WARN] {ch} {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                    continue
                
                # (AUC計算ロジックは変更なし)
                auc=roc_auc_score(y_true,y_score)
                auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
                fpr,tpr,thr=roc_curve(y_true,y_score)
                youden=tpr-fpr; idx=np.argmax(youden)
                cutoff_val = thr[idx]
                if np.isinf(cutoff_val) or np.isnan(cutoff_val):
                    DEBUG_LOG(f"[ERROR] {ch} {depth}mm {c1} vs {c2} ({f}): Cutoff is {cutoff_val}. Samples={n_samples_used}")

                rows.append({"Channel":ch,"Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                             "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})

# Ratio features (今回は時間特徴のみのため、Ratioは計算対象外)
# Time Ratio Features (e.g., RiseFall_ratio_Red/IR) は時間特徴ではないため、ここでは計算しない

df_auc_feat = pd.DataFrame(rows)
df_auc_feat.to_csv("AUC_time_features_bpm_norm.csv", index=False)
DEBUG_LOG("--- Part 4 Complete: AUC calculation for Time Features (BPM Norm) saved. ---")
print("✅ Time features AUC analysis (BPM Normalized) complete. Check AUC_time_features_bpm_norm.csv and log file.")

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import find_peaks, detrend, butter, filtfilt
from sklearn.metrics import roc_auc_score, roc_curve
import math
from scipy.fft import rfftfreq, rfft 

# ========= 基本設定 =========
fs = 2000  # Hz
depths = [3, 9, 15, 21]
state_map = {1: "Normal", 2: "Ischaemia", 3: "Congestion"}
channels = {"ppgA_Red_AGC": "Red", "ppgA_IR_AGC": "IR"} 
DEBUG_MODE = True
DEBUG_FILE = "debug_log_part1_to_3_amplitude_features.txt"
with open(DEBUG_FILE, "w") as f:
    f.write("--- Debug Log Start (Amplitude & Area Features) ---\n")

def DEBUG_LOG(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a") as f:
            f.write(message + "\n")

# --- BPF, BPM推定関数 (変更なし) ---
def bandpass_filter(sig, fs, low=0.5, high=5.0, order=3):
    nyq = fs / 2
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    return filtfilt(b, a, sig)

def estimate_bpm(signal, fs=2000):
    sig = pd.Series(signal, dtype=float).interpolate().fillna(0).values
    sig = detrend(sig)
    sig = bandpass_filter(sig, fs)
    prom = max(np.std(sig) * 0.3, np.ptp(sig) * 0.05)
    peaks, _ = find_peaks(sig, prominence=prom, distance=int(fs * 0.5))
    if len(peaks) < 2:
        return np.nan
    duration_sec = len(sig) / fs
    bpm = len(peaks) / (duration_sec / 60.0)
    return bpm

# --- ★★★ 1拍ごとの特徴抽出 (面積特徴量計算を追加) ★★★
def extract_pulse_features(signal, fs, dist=0.3):
    # ロバスト性向上のためのピーク検出閾値 (信号振幅の10%以上の高さを要求)
    threshold_height = np.min(signal) + (np.ptp(signal) * 0.1)
    peaks, _ = find_peaks(signal, height=threshold_height, distance=int(fs * dist))
    
    feats = []
    if len(peaks) < 2: return []
        
    for p in peaks:
        left = max(0, p - int(0.5 * fs))
        right = min(len(signal), p + int(0.5 * fs))
        seg = signal[left:right]
        if len(seg) < 5: continue
        if np.any(np.isnan(seg)) or np.all(seg == seg[0]): continue
        
        peak_idx = np.argmax(seg)
        trough_idx = np.argmin(seg[:peak_idx+1])
        next_trough_idx = peak_idx + np.argmin(seg[peak_idx:])
        
        if trough_idx < peak_idx < next_trough_idx:
            
            # --- 共通の基線と振幅の計算 ---
            baseline = seg[trough_idx]
            peak_val = seg[peak_idx]
            amp = peak_val - baseline
            
            # Pulse Area (全面積)
            full_seg = seg[trough_idx:next_trough_idx+1]
            # np.trapz(y, dx) で台形則による面積を計算
            area_pulse = np.trapz(full_seg - baseline, dx=1/fs) 
            
            # --- 面積特徴量の計算 (S-AUC, D-AUC) ---
            # S-AUC: 収縮期面積 (トラフからピークまで)
            sys_seg = seg[trough_idx:peak_idx+1]
            area_systolic = np.trapz(sys_seg - baseline, dx=1/fs)
            
            # D-AUC: 拡張期面積 (ピークから次のトラフまで)
            dias_seg = seg[peak_idx:next_trough_idx+1]
            area_diastolic = np.trapz(dias_seg - baseline, dx=1/fs)
            
            feats.append((area_pulse, area_systolic, area_diastolic, amp))
    return feats

# ========= Part 1 & 2: データロードと特徴抽出 =========
records = []
pulse_id_counter = 0 

for depth in depths:
    for i_state, state_name in state_map.items():
        file_path = Path(f"{depth}mm_AGC_data{i_state}.csv")
        DEBUG_LOG(f"\n--- Processing: {file_path.name} ({state_name}) ---")

        if not file_path.exists(): continue
        df = pd.read_csv(file_path)

        # --- BPM推定 ---
        ir_sig = df.get("ppgA_IR_AGC", pd.Series([]).values)
        if len(ir_sig) == 0: continue
            
        bpm = estimate_bpm(ir_sig, fs)
        window_sec = 60.0 / bpm if not np.isnan(bpm) and bpm > 0 else 1.0
        window_sec = float(np.clip(window_sec, 0.6, 2.0))
        
        step = int(round(fs * window_sec))
        current_pulse_id = pulse_id_counter 

        for col, ch_label in channels.items():
            if col not in df.columns: continue
            sig = df[col].values

            # (1) Window-based Amplitude (Amplitudeのサンプリングを増やす目的で維持)
            n_win = len(sig) // step
            DEBUG_LOG(f"  Channel: {ch_label}, Total Windows: {n_win}")
            
            for w in range(n_win):
                s, e = w * step, (w + 1) * step
                seg = sig[s:e]
                if len(seg) < 5: continue
                amp = seg.max() - seg.min()
                
                records.append({
                    "Depth": depth, "State": state_name, "Channel": ch_label,
                    "Pulse_ID": current_pulse_id,
                    "Amplitude": amp,
                    "Pulse_Area": np.nan, "S_AUC": np.nan, "D_AUC": np.nan
                })

            # (2) peaksで詳細特徴抽出 (Peak-based)
            feats = extract_pulse_features(sig, fs)
            DEBUG_LOG(f"  Channel: {ch_label}, Extracted Pulse Features: {len(feats)} pulses.")
            
            for area_pulse, area_systolic, area_diastolic, amp in feats:
                # 振幅と面積特徴量のみを記録
                records.append({
                    "Depth": depth, "State": state_name, "Channel": ch_label,
                    "Pulse_ID": current_pulse_id,
                    "Amplitude": amp,
                    "Pulse_Area": area_pulse,
                    "S_AUC": area_systolic,
                    "D_AUC": area_diastolic,
                })
        
        pulse_id_counter += 1 

df_feat = pd.DataFrame(records)
if df_feat.empty:
    raise RuntimeError("Error: No data loaded. Please check CSV files.")
DEBUG_LOG(f"\n--- Part 2 Complete: Total Feature Rows: {len(df_feat)} ---")


# ========= Part 3: 複合指標と正規化 (振幅・面積特徴に特化) =========
# Area Under Curve Ratio (Ratio between systolic and diastolic areas)
df_feat["AUC_Ratio"] = df_feat["S_AUC"] / (df_feat["D_AUC"] + 1e-6)


# --- 正規化（Redは3mm Red Normal基準、IRは3mm IR Normal基準） ---
# Amplitudeの正規化
def normalize_channel_amp(df, ch):
    base_data = df.query("Channel==@ch and Depth==3 and State=='Normal'")["Amplitude"]
    base = base_data.mean()
    DEBUG_LOG(f"Normalization Base (Amplitude, {ch}): Mean={base:.4e}, Count={len(base_data)}")
    
    if pd.isna(base) or base == 0:
        df.loc[df.Channel == ch, "Amp_norm"] = np.nan
    else:
        df.loc[df.Channel == ch, "Amp_norm"] = df.loc[df.Channel == ch, "Amplitude"] / base

normalize_channel_amp(df_feat, "Red")
normalize_channel_amp(df_feat, "IR")

# 面積特徴量の正規化
area_features = ["Pulse_Area", "S_AUC", "D_AUC"]

for f in area_features:
    def normalize_channel_area(df, ch, feature):
        base_data = df.query("Channel==@ch and Depth==3 and State=='Normal'")[feature]
        base = base_data.mean()
        
        DEBUG_LOG(f"Normalization Base ({feature}, {ch}): Mean={base:.4e}, Count={len(base_data)}")
        
        if pd.isna(base) or base == 0:
            df.loc[df.Channel == ch, f"{feature}_norm"] = np.nan
        else:
            df.loc[df.Channel == ch, f"{feature}_norm"] = df.loc[df.Channel == ch, feature] / base
            
    normalize_channel_area(df_feat, "Red", f)
    normalize_channel_area(df_feat, "IR", f)


# --- Red/IR比 ---
# Amplitudeと面積の正規化された比率を計算 (Red/IR_Ratio)

amp_data = df_feat.dropna(subset=['Amplitude', 'Amp_norm']).copy()

df_red = amp_data[amp_data.Channel == "Red"].rename(
    columns={"Amplitude": "Amplitude_Red", "Amp_norm": "Amp_norm_Red"}
)[['Depth', 'State', 'Pulse_ID', 'Amp_norm_Red'] + [f"{f}_norm" for f in area_features]]

df_ir = amp_data[amp_data.Channel == "IR"].rename(
    columns={"Amplitude": "Amplitude_IR", "Amp_norm": "Amp_norm_IR"}
)[['Depth', 'State', 'Pulse_ID', 'Amp_norm_IR'] + [f"{f}_norm" for f in area_features]]

df_ratio = pd.merge(df_red, df_ir, on=['Depth','State','Pulse_ID'], how='inner', suffixes=('_Red', '_IR'))
DEBUG_LOG(f"Ratio Data Rows (Merged for Amplitude/Area): {len(df_ratio)}")

# Red/IR Ratio 特徴量の計算
df_ratio["Red_IR_amp_norm_ratio"] = df_ratio["Amp_norm_Red"] / (df_ratio["Amp_norm_IR"] + 1e-6)

# ★★★ 修正箇所: 面積のRed/IR比の列名計算 (KeyError対策) ★★★
for f in area_features:
    # 列名に_normを付けて、評価リスト(ratio_features_to_eval)と一致させる
    df_ratio[f"{f}_norm_ratio"] = df_ratio[f"{f}_norm_Red"] / (df_ratio[f"{f}_norm_IR"] + 1e-6)
# ★★★ (修正完了) ★★★


# ========= Part 4: ROC AUC計算 (振幅・面積特徴に限定) =========
pairs = [("Normal","Ischaemia"),("Normal","Congestion"),("Ischaemia","Congestion")]

# 評価対象の特徴量リスト
features_to_eval = [
    "Amp_norm", "Pulse_Area_norm", "S_AUC_norm", "D_AUC_norm", "AUC_Ratio"
] 

# Red/IR Ratio 特徴量のリスト (修正後の列名を使用)
ratio_features_to_eval = [
    "Red_IR_amp_norm_ratio",
    "Pulse_Area_norm_ratio", "S_AUC_norm_ratio", "D_AUC_norm_ratio"
]

# AUC Bootstrap Helper
def bootstrap_auc_ci(y_true, y_score, n_bootstrap=200):
    rng = np.random.default_rng(42)
    aucs=[]
    n=len(y_true)
    for _ in range(n_bootstrap):
        idx = rng.integers(0,n,n)
        if len(np.unique(y_true[idx]))<2 or np.all(y_score[idx]==y_score[idx][0]): continue
        aucs.append(roc_auc_score(y_true[idx],y_score[idx]))
    if not aucs: return np.nan,np.nan,np.nan
    return np.mean(aucs), np.percentile(aucs,2.5), np.percentile(aucs,97.5)


rows=[]
# Channel-specific features
for ch in ["Red","IR"]:
    for depth in depths:
        sub = df_feat[(df_feat.Channel==ch)&(df_feat.Depth==depth)]
        if sub.empty: continue
        for f in features_to_eval:
            for c1,c2 in pairs:
                d=sub[sub.State.isin([c1,c2])]
                if d.empty: continue
                y_true=(d.State==c1).astype(int).values
                y_score=d[f].values
                mask=~np.isnan(y_score)
                y_true=y_true[mask]; y_score=y_score[mask]
                
                n_samples_used = len(y_true)
                if n_samples_used < 5 or len(np.unique(y_true))<2: 
                    DEBUG_LOG(f"[WARN] {ch} {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                    continue
                
                auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
                rows.append({"Channel":ch,"Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                             "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})

# Ratio features
for depth in depths:
    sub=df_ratio[df_ratio.Depth==depth]
    if sub.empty: continue
    for f in ratio_features_to_eval:
        for c1,c2 in pairs:
            d=sub[sub.State.isin([c1,c2])]
            if d.empty: continue
            y_true=(d.State==c1).astype(int).values
            y_score=d[f].values
            mask=~np.isnan(y_score)
            y_true=y_true[mask]; y_score=y_score[mask]
            
            n_samples_used = len(y_true)
            if n_samples_used < 5 or len(np.unique(y_true))<2: 
                DEBUG_LOG(f"[WARN] Ratio {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                continue
            
            auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
            rows.append({"Channel":"Red/IR_Ratio","Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                         "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})


df_auc_feat = pd.DataFrame(rows)
df_auc_feat.to_csv("AUC_amplitude_area_features.csv", index=False)
DEBUG_LOG("--- Part 4 Complete: AUC calculation for Amplitude/Area Features saved. ---")
print("✅ Amplitude/Area features AUC analysis complete. Check AUC_amplitude_area_features.csv and log file.")

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import find_peaks, detrend, butter, filtfilt
from sklearn.metrics import roc_auc_score, roc_curve
import math
from scipy.fft import rfftfreq, rfft 

# ========= 基本設定 =========
fs = 2000  # Hz
depths = [3, 9, 15, 21]
state_map = {1: "Normal", 2: "Ischaemia", 3: "Congestion"}
channels = {"ppgA_Red_AGC": "Red", "ppgA_IR_AGC": "IR"} 
DEBUG_MODE = True
DEBUG_FILE = "debug_log_part1_to_3_slope_features.txt"
with open(DEBUG_FILE, "w") as f:
    f.write("--- Debug Log Start (Slope & Morphology Features) ---\n")

def DEBUG_LOG(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a") as f:
            f.write(message + "\n")

# --- BPF, BPM推定関数 (変更なし) ---
def bandpass_filter(sig, fs, low=0.5, high=5.0, order=3):
    nyq = fs / 2
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    return filtfilt(b, a, sig)

def estimate_bpm(signal, fs=2000):
    sig = pd.Series(signal, dtype=float).interpolate().fillna(0).values
    sig = detrend(sig)
    sig = bandpass_filter(sig, fs)
    prom = max(np.std(sig) * 0.3, np.ptp(sig) * 0.05)
    peaks, _ = find_peaks(sig, prominence=prom, distance=int(fs * 0.5))
    if len(peaks) < 2:
        return np.nan
    duration_sec = len(sig) / fs
    bpm = len(peaks) / (duration_sec / 60.0)
    return bpm

# --- ★★★ 1拍ごとの特徴抽出 (勾配・形状特徴量計算を追加) ★★★
def extract_pulse_features(signal, fs, dist=0.3):
    threshold_height = np.min(signal) + (np.ptp(signal) * 0.1)
    peaks, _ = find_peaks(signal, height=threshold_height, distance=int(fs * dist))
    
    feats = []
    if len(peaks) < 2: return []
        
    for p in peaks:
        left = max(0, p - int(0.5 * fs))
        right = min(len(signal), p + int(0.5 * fs))
        seg = signal[left:right]
        if len(seg) < 5: continue
        if np.any(np.isnan(seg)) or np.all(seg == seg[0]): continue
        
        peak_idx = np.argmax(seg)
        trough_idx = np.argmin(seg[:peak_idx+1])
        next_trough_idx = peak_idx + np.argmin(seg[peak_idx:])
        
        if trough_idx < peak_idx < next_trough_idx:
            
            # 共通の特徴
            baseline = seg[trough_idx]
            peak_val = seg[peak_idx]
            amp = peak_val - baseline
            rise_time_s = (peak_idx - trough_idx) / fs
            fall_time_s = (next_trough_idx - peak_idx) / fs
            
            # --- 勾配特徴 ---
            # Upslope: Amp / Rise Time
            upslope = amp / (rise_time_s + 1e-6)
            # Downslope: -Amp / Fall Time (負の値になるため絶対値または正の値で定義)
            downslope = np.abs(amp / (fall_time_s + 1e-6))
            
            # --- 長さ特徴 (ユークリッド距離) ---
            # 信号のサンプリング間隔 (dx)
            dx = 1/fs
            
            # Upslope Length (UL): トラフからピークまでのユークリッド距離
            rise_seg = seg[trough_idx:peak_idx+1]
            dy_rise = np.diff(rise_seg)
            ul = np.sum(np.sqrt(dy_rise**2 + dx**2))
            
            # Downslope Length (DL): ピークから次のトラフまでのユークリッド距離
            fall_seg = seg[peak_idx:next_trough_idx+1]
            dy_fall = np.diff(fall_seg)
            dl = np.sum(np.sqrt(dy_fall**2 + dx**2))
            
            # --- Pulse Width (PW) ---
            half = baseline + 0.5 * amp
            seg_pw = seg[trough_idx:next_trough_idx+1]
            above_half = np.where(seg_pw >= half)[0]
            pw = (above_half[-1] - above_half[0]) / fs if len(above_half) > 1 else np.nan
            
            feats.append((amp, pw, rise_time_s, fall_time_s, upslope, downslope, ul, dl))
    return feats

# ========= Part 1 & 2: データロードと特徴抽出 =========
records = []
pulse_id_counter = 0 

for depth in depths:
    for i_state, state_name in state_map.items():
        file_path = Path(f"{depth}mm_AGC_data{i_state}.csv")
        DEBUG_LOG(f"\n--- Processing: {file_path.name} ({state_name}) ---")

        if not file_path.exists(): continue
        df = pd.read_csv(file_path)

        # --- BPM推定 ---
        ir_sig = df.get("ppgA_IR_AGC", pd.Series([]).values)
        if len(ir_sig) == 0: continue
            
        bpm = estimate_bpm(ir_sig, fs)
        window_sec = 60.0 / bpm if not np.isnan(bpm) and bpm > 0 else 1.0
        window_sec = float(np.clip(window_sec, 0.6, 2.0))
        
        step = int(round(fs * window_sec))
        current_pulse_id = pulse_id_counter 

        for col, ch_label in channels.items():
            if col not in df.columns: continue
            sig = df[col].values

            # (1) Window-based Amplitude 
            n_win = len(sig) // step
            DEBUG_LOG(f"  Channel: {ch_label}, Total Windows: {n_win}")
            
            for w in range(n_win):
                s, e = w * step, (w + 1) * step
                seg = sig[s:e]
                if len(seg) < 5: continue
                amp = seg.max() - seg.min()
                
                records.append({
                    "Depth": depth, "State": state_name, "Channel": ch_label,
                    "Pulse_ID": current_pulse_id,
                    "Amplitude": amp, "PulseWidth": np.nan, 
                    "Upslope": np.nan, "Downslope": np.nan, "UpslopeLength": np.nan, "DownslopeLength": np.nan
                })

            # (2) peaksで詳細特徴抽出 (Peak-based)
            # ★修正: 拡張した特徴量リストを受け取る★
            feats = extract_pulse_features(sig, fs)
            DEBUG_LOG(f"  Channel: {ch_label}, Extracted Pulse Features: {len(feats)} pulses.")
            
            for amp, pw, rt, ft, us, ds, ul, dl in feats:
                records.append({
                    "Depth": depth, "State": state_name, "Channel": ch_label,
                    "Pulse_ID": current_pulse_id,
                    "Amplitude": amp,
                    "PulseWidth": pw,
                    "Upslope": us, 
                    "Downslope": ds,
                    "UpslopeLength": ul,
                    "DownslopeLength": dl
                })
        
        pulse_id_counter += 1 

df_feat = pd.DataFrame(records)
if df_feat.empty:
    raise RuntimeError("Error: No data loaded. Please check CSV files.")
DEBUG_LOG(f"\n--- Part 2 Complete: Total Feature Rows: {len(df_feat)} ---")


# ========= Part 3: 複合指標と正規化 (勾配・形状特徴に特化) =========

# --- 正規化（Redは3mm Red Normal基準、IRは3mm IR Normal基準） ---
def normalize_channel_amp(df, ch):
    base_data = df.query("Channel==@ch and Depth==3 and State=='Normal'")["Amplitude"]
    base = base_data.mean()
    DEBUG_LOG(f"Normalization Base (Amplitude, {ch}): Mean={base:.4e}, Count={len(base_data)}")
    
    if pd.isna(base) or base == 0:
        df.loc[df.Channel == ch, "Amp_norm"] = np.nan
    else:
        df.loc[df.Channel == ch, "Amp_norm"] = df.loc[df.Channel == ch, "Amplitude"] / base

normalize_channel_amp(df_feat, "Red")
normalize_channel_amp(df_feat, "IR")

# ★★★ 勾配・形状の特徴量計算 ★★★
# Amplitude_normを基準に正規化が必要な特徴量の列名
features_for_norm = ["Upslope", "Downslope", "UpslopeLength", "DownslopeLength"]

for f in features_for_norm:
    # 勾配（Upslope, Downslope）や長さ（UL, DL）は振幅に依存するため、Amp_normで正規化
    df_feat[f"{f}_norm"] = df_feat[f] / (df_feat["Amp_norm"] + 1e-6)

# ★★★ 複合特徴量 ★★★
df_feat["Slope_Ratio"] = df_feat["Upslope_norm"] / (df_feat["Downslope_norm"] + 1e-6) # US_norm / DS_norm
df_feat["Length_Height_Ratio"] = df_feat["PulseWidth"] / (df_feat["Amplitude"] + 1e-6) # Width / Amp
df_feat["SlopeLength_Ratio"] = df_feat["UpslopeLength_norm"] / (df_feat["DownslopeLength_norm"] + 1e-6) # UL_norm / DL_norm

DEBUG_LOG(f"\n--- Part 3 Complete: Normalized Slope Features Created ---")

# --- Red/IR比 (マージロジック) ---
amp_data = df_feat.dropna(subset=['Amplitude', 'Amp_norm']).copy()

merge_cols = ['Depth', 'State', 'Pulse_ID'] + features_for_norm + [f"{f}_norm" for f in features_for_norm] + ["Amplitude", "Amp_norm", "PulseWidth"]

df_red = amp_data[amp_data.Channel == "Red"][merge_cols].rename(columns={"Amplitude": "Amplitude_Red", "Amp_norm": "Amp_norm_Red"})
df_ir = amp_data[amp_data.Channel == "IR"][merge_cols].rename(columns={"Amplitude": "Amplitude_IR", "Amp_norm": "Amp_norm_IR"})

df_ratio = pd.merge(df_red, df_ir, on=['Depth','State','Pulse_ID'], how='inner', suffixes=('_Red', '_IR'))
DEBUG_LOG(f"Ratio Data Rows (Merged for Slope Features): {len(df_ratio)}")

# Red/IR Ratio 勾配・長さの特徴量の計算
# Slope/Length の Red/IR 比率（例: Red_Upslope_norm / IR_Upslope_norm）
for f in features_for_norm:
    df_ratio[f"{f}_norm_ratio"] = df_ratio[f"{f}_norm_Red"] / (df_ratio[f"{f}_norm_IR"] + 1e-6)

# Amplitude Ratioは評価対象外のため省略

# ========= Part 4: ROC AUC計算 (勾配・形状特徴に限定) =========
pairs = [("Normal","Ischaemia"),("Normal","Congestion"),("Ischaemia","Congestion")]

# ★★★ 評価対象の特徴量リストを更新 ★★★
features_to_eval = [
    "Upslope_norm", "Downslope_norm", "UpslopeLength_norm", "DownslopeLength_norm", 
    "Slope_Ratio", "Length_Height_Ratio", "SlopeLength_Ratio"
] 

# Red/IR Ratio 勾配・長さの特徴量のリスト
ratio_features_to_eval = [
    f"{f}_norm_ratio" for f in features_for_norm
]

# AUC Bootstrap Helper (再定義は省略)
def bootstrap_auc_ci(y_true, y_score, n_bootstrap=200):
    rng = np.random.default_rng(42)
    aucs=[]
    n=len(y_true)
    for _ in range(n_bootstrap):
        idx = rng.integers(0,n,n)
        if len(np.unique(y_true[idx]))<2 or np.all(y_score[idx]==y_score[idx][0]): continue
        aucs.append(roc_auc_score(y_true[idx],y_score[idx]))
    if not aucs: return np.nan,np.nan,np.nan
    return np.mean(aucs), np.percentile(aucs,2.5), np.percentile(aucs,97.5)


rows=[]
# Channel-specific features
for ch in ["Red","IR"]:
    for depth in depths:
        sub = df_feat[(df_feat.Channel==ch)&(df_feat.Depth==depth)]
        if sub.empty: continue
        for f in features_to_eval:
            for c1,c2 in pairs:
                d=sub[sub.State.isin([c1,c2])]
                if d.empty: continue
                y_true=(d.State==c1).astype(int).values
                y_score=d[f].values
                mask=~np.isnan(y_score)
                y_true=y_true[mask]; y_score=y_score[mask]
                
                n_samples_used = len(y_true)
                if n_samples_used < 5 or len(np.unique(y_true))<2: 
                    DEBUG_LOG(f"[WARN] {ch} {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                    continue
                
                auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
                rows.append({"Channel":ch,"Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                             "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})

# Ratio features
for depth in depths:
    sub=df_ratio[df_ratio.Depth==depth]
    if sub.empty: continue
    for f in ratio_features_to_eval:
        for c1,c2 in pairs:
            d=sub[sub.State.isin([c1,c2])]
            if d.empty: continue
            y_true=(d.State==c1).astype(int).values
            y_score=d[f].values
            mask=~np.isnan(y_score)
            y_true=y_true[mask]; y_score=y_score[mask]
            
            n_samples_used = len(y_true)
            if n_samples_used < 5 or len(np.unique(y_true))<2: 
                DEBUG_LOG(f"[WARN] Ratio {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                continue
            
            auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
            rows.append({"Channel":"Red/IR_Ratio","Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                         "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})


df_auc_feat = pd.DataFrame(rows)
df_auc_feat.to_csv("AUC_slope_morphology_features.csv", index=False)
DEBUG_LOG("--- Part 4 Complete: AUC calculation for Slope/Morphology Features saved. ---")
print("✅ Slope/Morphology features AUC analysis complete. Check AUC_slope_morphology_features.csv and log file.")

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import find_peaks, detrend, butter, filtfilt
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats import skew, kurtosis 
import math
from scipy.fft import rfftfreq, rfft 

# ========= 基本設定 =========
fs = 2000  # Hz
depths = [3, 9, 15, 21]
state_map = {1: "Normal", 2: "Ischaemia", 3: "Congestion"}
channels = {"ppgA_Red_AGC": "Red", "ppgA_IR_AGC": "IR"} 
DEBUG_MODE = True
DEBUG_FILE = "debug_log_part1_to_3_composite_noise_features.txt"
with open(DEBUG_FILE, "w") as f:
    f.write("--- Debug Log Start (Composite & Noise Features) ---\n")

def DEBUG_LOG(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a") as f:
            f.write(message + "\n")

# --- BPF, BPM推定関数 (変更なし) ---
def bandpass_filter(sig, fs, low=0.5, high=5.0, order=3):
    nyq = fs / 2
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    return filtfilt(b, a, sig)

def estimate_bpm(signal, fs=2000):
    sig = pd.Series(signal, dtype=float).interpolate().fillna(0).values
    sig = detrend(sig)
    sig = bandpass_filter(sig, fs)
    prom = max(np.std(sig) * 0.3, np.ptp(sig) * 0.05)
    peaks, _ = find_peaks(sig, prominence=prom, distance=int(fs * 0.5))
    if len(peaks) < 2:
        return np.nan
    duration_sec = len(sig) / fs
    bpm = len(peaks) / (duration_sec / 60.0)
    return bpm

# --- ★★★ 1拍ごとの特徴抽出 (歪度・尖度・Width Ratioの準備) ★★★
def extract_pulse_features(signal, fs, dist=0.3):
    threshold_height = np.min(signal) + (np.ptp(signal) * 0.1)
    peaks, _ = find_peaks(signal, height=threshold_height, distance=int(fs * dist))
    
    feats = []
    if len(peaks) < 2: return []
        
    for p in peaks:
        left = max(0, p - int(0.5 * fs))
        right = min(len(signal), p + int(0.5 * fs))
        seg = signal[left:right]
        if len(seg) < 5: continue
        if np.any(np.isnan(seg)) or np.all(seg == seg[0]): continue
        
        peak_idx = np.argmax(seg)
        trough_idx = np.argmin(seg[:peak_idx+1])
        next_trough_idx = peak_idx + np.argmin(seg[peak_idx:])
        
        if trough_idx < peak_idx < next_trough_idx:
            
            baseline = seg[trough_idx]
            amp = seg[peak_idx] - baseline
            
            # --- 複合・ノイズ特徴 (1拍セグメントベース) ---
            pulse_seg = seg[trough_idx:next_trough_idx+1] - baseline # 基線補正
            
            # 1. Skewness (歪度)
            seg_skew = skew(pulse_seg)
            
            # 2. Kurtosis (尖度)
            seg_kurtosis = kurtosis(pulse_seg)
            
            # 3. Pulse Width (PW) - Width Ratioの準備
            half = baseline + 0.5 * amp
            seg_pw_full = seg[trough_idx:next_trough_idx+1]
            above_half = np.where(seg_pw_full >= half)[0]
            # 拍周期
            period_s = (next_trough_idx - trough_idx) / fs
            # パルス幅
            pw_s = (above_half[-1] - above_half[0]) / fs if len(above_half) > 1 else np.nan
            
            # 4. Datum Area (基線からの総面積) - Area特徴と重複するため、ここではPulse Areaとして計算
            area_datum = np.trapz(pulse_seg, dx=1/fs) 
            
            feats.append((amp, seg_skew, seg_kurtosis, pw_s, period_s, area_datum))
    return feats

# --- 信号全体の特徴量計算 (SNR, ZCR) ---
def calculate_global_features(sig, fs):
    # 1. SNR (Signal-to-Noise Ratio)
    # PPGではDC成分(トレンド)を信号と見なし、残りの高周波成分をノイズと見なす方法が一般的
    
    # 信号成分 (低周波成分) - BPFを通すことでノイズと分離
    signal_component = bandpass_filter(sig, fs)
    
    # ノイズ成分 (信号からBPF成分を引いたもの) - 高周波ノイズ＋DCトレンド
    # DCトレンドを除去するためにdetrend済み信号を使用
    detrended_sig = detrend(sig)
    noise_component = detrended_sig - signal_component

    # SNR in dB
    power_signal = np.mean(signal_component**2)
    power_noise = np.mean(noise_component**2)
    
    if power_noise > 0:
        snr_db = 10 * np.log10(power_signal / power_noise)
    else:
        snr_db = np.nan
        
    # 2. ZCR (Zero-Crossing Rate) - 信号の周波数・ノイズレベルの指標
    # Detrended (平均ゼロ付近)の信号を使用
    zcr = np.sum(np.diff(np.sign(detrended_sig)) != 0) / len(detrended_sig)
    
    return snr_db, zcr


# ========= Part 1 & 2: データロードと特徴抽出 =========
records = []
global_records = []
pulse_id_counter = 0 

for depth in depths:
    for i_state, state_name in state_map.items():
        file_path = Path(f"{depth}mm_AGC_data{i_state}.csv")
        DEBUG_LOG(f"\n--- Processing: {file_path.name} ({state_name}) ---")

        if not file_path.exists(): continue
        df = pd.read_csv(file_path)

        # --- BPM推定 ---
        ir_sig = df.get("ppgA_IR_AGC", pd.Series([]).values)
        if len(ir_sig) == 0: continue
            
        bpm = estimate_bpm(ir_sig, fs)
        window_sec = 60.0 / bpm if not np.isnan(bpm) and bpm > 0 else 1.0
        window_sec = float(np.clip(window_sec, 0.6, 2.0))
        
        step = int(round(fs * window_sec))
        current_pulse_id = pulse_id_counter 

        for col, ch_label in channels.items():
            if col not in df.columns: continue
            sig = df[col].values

            # --- (A) 全体特徴量 (SNR, ZCR) の計算と記録 ---
            snr, zcr = calculate_global_features(sig, fs)
            global_records.append({
                "Depth": depth, "State": state_name, "Channel": ch_label,
                "Pulse_ID": current_pulse_id, # このIDでマージする
                "SNR": snr, "ZCR": zcr
            })
            DEBUG_LOG(f"  Channel: {ch_label}, SNR: {snr:.2f} dB, ZCR: {zcr:.4f}")

            # --- (B) 1拍ベースの特徴抽出 (歪度, 尖度, 幅, 面積) ---
            feats = extract_pulse_features(sig, fs)
            DEBUG_LOG(f"  Channel: {ch_label}, Extracted Pulse Features: {len(feats)} pulses.")
            
            for amp, seg_skew, seg_kurtosis, pw_s, period_s, area_datum in feats:
                records.append({
                    "Depth": depth, "State": state_name, "Channel": ch_label,
                    "Pulse_ID": current_pulse_id, # Global特徴量と結合するためのID
                    "Amplitude": amp,
                    "Skewness": seg_skew,
                    "Kurtosis": seg_kurtosis,
                    "PulseWidth": pw_s,
                    "PulsePeriod": period_s,
                    "DatumArea": area_datum,
                })
        
        pulse_id_counter += 1 

df_feat_pulse = pd.DataFrame(records)
df_feat_global = pd.DataFrame(global_records)
if df_feat_pulse.empty or df_feat_global.empty:
    raise RuntimeError("Error: No data loaded. Please check CSV files.")

# Global特徴量をPulse特徴量に結合
df_feat = pd.merge(df_feat_pulse, df_feat_global, on=['Depth','State','Channel','Pulse_ID'], how='left')

DEBUG_LOG(f"\n--- Part 2 Complete: Total Feature Rows: {len(df_feat)} ---")


# ========= Part 3: 複合指標と正規化 (複合・ノイズ特徴に特化) =========

# --- 複合特徴量 (比率) ---
# Width Ratio: Pulse Width / Pulse Period
df_feat["Width_Ratio"] = df_feat["PulseWidth"] / (df_feat["PulsePeriod"] + 1e-6)

# Amplitudeの正規化
def normalize_channel_amp(df, ch):
    base_data = df.query("Channel==@ch and Depth==3 and State=='Normal'")["Amplitude"]
    base = base_data.mean()
    DEBUG_LOG(f"Normalization Base (Amplitude, {ch}): Mean={base:.4e}, Count={len(base_data)}")
    
    if pd.isna(base) or base == 0:
        df.loc[df.Channel == ch, "Amp_norm"] = np.nan
    else:
        df.loc[df.Channel == ch, "Amp_norm"] = df.loc[df.Channel == ch, "Amplitude"] / base

normalize_channel_amp(df_feat, "Red")
normalize_channel_amp(df_feat, "IR")

# Datum AreaをAmp_normで正規化 (面積は振幅に依存するため)
df_feat["DatumArea_norm"] = df_feat["DatumArea"] / (df_feat["Amp_norm"] + 1e-6)

DEBUG_LOG(f"\n--- Part 3 Complete: Composite Features Created ---")


# --- Red/IR比 (マージロジック) ---
# 歪度、尖度、Width Ratio、SNR、ZCRは比率にしない（または意味がない）
# Datum Area_norm の Red/IR 比を計算する

amp_data = df_feat.dropna(subset=['Amplitude', 'Amp_norm']).copy()

# 比率計算の対象となる特徴量（正規化済みDatum Area）
area_features = ["DatumArea"] 
# Width Ratioは、RedとIRで脈拍周期が等しいと仮定すると、Pulse Widthの比率と等価

merge_cols = ['Depth', 'State', 'Pulse_ID', 'DatumArea_norm']

df_red = amp_data[amp_data.Channel == "Red"][merge_cols].rename(
    columns={"DatumArea_norm": "DatumArea_norm_Red"}
)
df_ir = amp_data[amp_data.Channel == "IR"][merge_cols].rename(
    columns={"DatumArea_norm": "DatumArea_norm_IR"}
)

df_ratio = pd.merge(df_red, df_ir, on=['Depth','State','Pulse_ID'], how='inner')
DEBUG_LOG(f"Ratio Data Rows (Merged for Datum Area Ratio): {len(df_ratio)}")

# Datum Area Ratio の計算
df_ratio["DatumArea_norm_ratio"] = df_ratio["DatumArea_norm_Red"] / (df_ratio["DatumArea_norm_IR"] + 1e-6)


# ========= Part 4: ROC AUC計算 (複合・ノイズ特徴に限定) =========
pairs = [("Normal","Ischaemia"),("Normal","Congestion"),("Ischaemia","Congestion")]

# ★★★ 評価対象の特徴量リストを更新 ★★★
# Skewness, Kurtosis, Width_Ratio, SNR, ZCR は単一チャンネル特徴量
features_to_eval = [
    "Skewness", "Kurtosis", "Width_Ratio", "SNR", "ZCR", "DatumArea_norm"
] 

# Red/IR Ratio 特徴量のリスト
ratio_features_to_eval = [
    "DatumArea_norm_ratio"
]

# AUC Bootstrap Helper
def bootstrap_auc_ci(y_true, y_score, n_bootstrap=200):
    rng = np.random.default_rng(42)
    aucs=[]
    n=len(y_true)
    for _ in range(n_bootstrap):
        idx = rng.integers(0,n,n)
        if len(np.unique(y_true[idx]))<2 or np.all(y_score[idx]==y_score[idx][0]): continue
        aucs.append(roc_auc_score(y_true[idx],y_score[idx]))
    if not aucs: return np.nan,np.nan,np.nan
    return np.mean(aucs), np.percentile(aucs,2.5), np.percentile(aucs,97.5)


rows=[]
# Channel-specific features
for ch in ["Red","IR"]:
    for depth in depths:
        sub = df_feat[(df_feat.Channel==ch)&(df_feat.Depth==depth)]
        if sub.empty: continue
        for f in features_to_eval:
            for c1,c2 in pairs:
                # SNR, ZCRはPulse_ID単位でユニークなため、重複を削除してから評価する
                if f in ["SNR", "ZCR"]:
                    d = sub[sub.State.isin([c1,c2])].drop_duplicates(subset=["Pulse_ID"])
                else:
                    d = sub[sub.State.isin([c1,c2])]
                    
                if d.empty: continue
                
                y_true=(d.State==c1).astype(int).values
                y_score=d[f].values
                mask=~np.isnan(y_score)
                y_true=y_true[mask]; y_score=y_score[mask]
                
                n_samples_used = len(y_true)
                if n_samples_used < 5 or len(np.unique(y_true))<2: 
                    DEBUG_LOG(f"[WARN] {ch} {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                    continue
                
                auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
                rows.append({"Channel":ch,"Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                             "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})

# Ratio features
for depth in depths:
    sub=df_ratio[df_ratio.Depth==depth]
    if sub.empty: continue
    for f in ratio_features_to_eval:
        for c1,c2 in pairs:
            d=sub[sub.State.isin([c1,c2])]
            if d.empty: continue
            y_true=(d.State==c1).astype(int).values
            y_score=d[f].values
            mask=~np.isnan(y_score)
            y_true=y_true[mask]; y_score=y_score[mask]
            
            n_samples_used = len(y_true)
            if n_samples_used < 5 or len(np.unique(y_true))<2: 
                DEBUG_LOG(f"[WARN] Ratio {depth}mm {c1} vs {c2} ({f}): Insufficient samples ({n_samples_used}) or classes.")
                continue
            
            auc_bs,ci_lo,ci_hi=bootstrap_auc_ci(y_true,y_score)
            rows.append({"Channel":"Red/IR_Ratio","Depth":depth,"Feature":f,"Pair":f"{c1} vs {c2}",
                         "AUC_bootstrap_mean":auc_bs,"CI_lower":ci_lo,"CI_upper":ci_hi,"n_samples":n_samples_used})


df_auc_feat = pd.DataFrame(rows)
df_auc_feat.to_csv("AUC_composite_noise_features.csv", index=False)
DEBUG_LOG("--- Part 4 Complete: AUC calculation for Composite/Noise Features saved. ---")
print("✅ Composite/Noise features AUC analysis complete. Check AUC_composite_noise_features.csv and log file.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# ========= 1. データ読み込みと準備 =========

# 最終的に生成されたAUCサマリーファイル名を指定（必要に応じて変更してください）
FILE_PATH = "AUC_all.csv"
try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}. Please run the previous code block to generate the CSV.")
    exit()

# NaN値を含む行を削除し、AUC値が0.7未満のものは無視する
df_filtered = df.dropna(subset=['AUC_bootstrap_mean'])
df_filtered_plot = df_filtered[df_filtered['AUC_bootstrap_mean'] > 0.7].copy() # プロット対象（AUC>0.7）のみ

# ========= 2. 特徴量の整形と色の定義 =========

# 2.1. 特徴名のマッピングと結合
def format_feature_name(row):
    feature = row['Feature']
    channel = row['Channel']
    
    # Amp_normをAmplitudeに改名
    if feature == 'Amp_norm':
        feature = 'Amplitude'
        
    # Pulse_Area_normはPulse_Areaと同一とみなし削除（Noneを返すことでフィルタリング）
    if feature == 'Pulse_Area_norm':
        return None
        
    # ChannelとFeatureを結合 
    if channel in ['Red', 'IR']:
        return f"{channel}_{feature}"
    elif channel == 'Red/IR_Ratio':
        # Ratioの場合はFeature名をそのまま使用（例: Red_IR_amp_norm_ratio）
        return feature
    return None

df_filtered_plot['Combined_Feature'] = df_filtered_plot.apply(format_feature_name, axis=1)
df_filtered_plot = df_filtered_plot.dropna(subset=['Combined_Feature'])

# 2.2. 色の定義
def get_feature_color(feature_name):
    """Red/IRに基づいて色を決定"""
    feature_name = feature_name.lower()
    if 'red' in feature_name:
        return 'red'
    elif 'ir' in feature_name:
        return 'purple'
    return 'black'

# 2.3. ヒートマップ描画用データフレームの準備
pair_order = [
    'Normal vs Ischaemia',
    'Normal vs Congestion',
    'Ischaemia vs Congestion'
]
depth_order = sorted(df_filtered_plot['Depth'].unique())

text_grid = {} # key: (Pair, Depth), value: [(text, color, auc_value), ...]
heatmap_values = pd.DataFrame(index=pair_order, columns=depth_order, dtype=float)

for index, row in df_filtered_plot.iterrows():
    pair = row['Pair']
    depth = row['Depth']
    feature_name = row['Combined_Feature']
    auc_value = row['AUC_bootstrap_mean']
    
    key = (pair, depth)
    
    # テキスト準備
    text = f"{feature_name}: {auc_value:.2f}"
    color = get_feature_color(feature_name)
    
    if key not in text_grid:
        text_grid[key] = []
    # (text, color, auc_value) のタプルでAUC値も格納
    text_grid[key].append((text, color, auc_value))
    
    # 背景色の基準（セル内の最大AUC）を更新
    if pair in pair_order and depth in depth_order:
        current_max = heatmap_values.loc[pair, depth]
        if pd.isna(current_max) or auc_value > current_max:
            heatmap_values.loc[pair, depth] = auc_value


# AUCが0.7未満（プロット対象外）のセルは、カラーマップの最小値（0.5）で埋める
MIN_AUC_FOR_CMAP = 0.5 
heatmap_values = heatmap_values.fillna(MIN_AUC_FOR_CMAP) 

# ========= 3. ヒートマップのプロット =========

fig, ax = plt.subplots(figsize=(13, 4)) 

# 3.1. 背景色
cmap = plt.cm.get_cmap('YlGnBu_r') 
norm = mcolors.Normalize(vmin=MIN_AUC_FOR_CMAP, vmax=1.0) 

cax = ax.imshow(heatmap_values.values, cmap=cmap, norm=norm, aspect='auto', interpolation='nearest')

# 3.2. 軸の設定
ax.set_xticks(np.arange(len(depth_order)))
ax.set_yticks(np.arange(len(pair_order)))
ax.set_xticklabels([f'{d} mm' for d in depth_order])
ax.set_yticklabels(pair_order)

ax.set_xlabel('Depth (mm)', fontsize=13)
ax.set_ylabel('Pairwise Comparison', fontsize=13)
ax.set_title(f'High AUC Features (Top 5)', fontsize=16, pad=20)

# 3.3. テキストの描画
for i, pair in enumerate(pair_order):
    for j, depth in enumerate(depth_order):
        key = (pair, depth)
        
        # AUC > 0.7のデータがあるセルのみにテキストを描画
        if key in text_grid and heatmap_values.loc[pair, depth] > 0.7:
            
            # ★★★ 修正箇所: AUCで降順ソートし、上位5つに制限 ★★★
            # text_grid[key] は [(text, color, auc_value), ...]
            sorted_entries = sorted(text_grid[key], key=lambda x: x[2], reverse=True)
            
            # 上位5つの (text, color) のみを取得
            top_5_entries = [(t, c) for t, c, a in sorted_entries[:5]] 
            
            n_entries = len(top_5_entries)
            
            # 垂直方向の中心から上下にずらして配置を調整
            # 5行まで対応するようにオフセットを調整
            y_offsets = np.linspace(-0.35, 0.35, n_entries) if n_entries > 0 else []
            
            for k, (text, color) in enumerate(top_5_entries):
                ax.text(j, i + y_offsets[k],
                        text, 
                        ha="center", 
                        va="center", 
                        color=color, 
                        fontsize=8, 
                        linespacing=1.2)

# 3.4. カラーバーの追加と修正
cbar_ticks = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
cbar = fig.colorbar(cax, ticks=cbar_ticks, label='AUC Value')

custom_labels = [f'{t:.1f}' for t in cbar_ticks]
custom_labels[0] = f'≤ {MIN_AUC_FOR_CMAP:.1f}'

cbar.ax.set_yticklabels(custom_labels) 

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import os

# 1. ファイルパスのリストを定義
file_paths = [
    "AUC_time_features_bpm_norm.csv",
    "AUC_amplitude_area_features.csv",
    "AUC_slope_morphology_features.csv",
    "AUC_composite_noise_features.csv"
]

all_data = []

# 2. 各CSVファイルを読み込み、リストに追加
for file_path in file_paths:
    try:
        df = pd.read_csv(file_path)
        all_data.append(df)
        print(f"✅ Loaded: {file_path}")
    except FileNotFoundError:
        # ファイルが見つからない場合は警告を表示してスキップ
        print(f"❌ Warning: File not found: {file_path}. Skipping this file.")
    except Exception as e:
        print(f"❌ Error reading {file_path}: {e}. Skipping this file.")
        
# 3. 全てのDataFrameを結合
if not all_data:
    print("\n⚠️ Error: No dataframes were loaded. Check file paths.")
else:
    # 結合 (indexをリセット)
    df_combined = pd.concat(all_data, ignore_index=True)

    # 4. 結合したDataFrameをAUC_all.csvとして保存
    output_file = "AUC_all.csv"
    df_combined.to_csv(output_file, index=False)

    print(f"\n--- Output created ---")
    print(f"✅ All AUC results combined into {output_file}.")
    print(f"Total rows in {output_file}: {len(df_combined)}")

    # 結合されたデータの一部を表示
    print("\n--- AUC_all.csv Head (Sample Data) ---")
    print(df_combined.head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# ========= 1. データ読み込みと準備 =========

# 最終的に生成されたAUCサマリーファイル名を指定（必要に応じて変更してください）
FILE_PATH = "AUC_all.csv"
try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}. Please run the previous code block to generate the CSV.")
    exit()

# NaN値を含む行を削除し、AUC値が0.7未満のものは無視する
df_filtered = df.dropna(subset=['AUC_bootstrap_mean'])
df_filtered_plot = df_filtered[df_filtered['AUC_bootstrap_mean'] > 0.7].copy() # プロット対象（AUC>0.7）のみ

# ========= 2. 特徴量の整形と色の定義 =========

# 2.1. 特徴名のマッピングと結合
def format_feature_name(row):
    feature = row['Feature']
    channel = row['Channel']
    
    # Amp_normをAmplitudeに改名
    if feature == 'Amp_norm':
        feature = 'Amplitude'
        
    # Pulse_Area_normはPulse_Areaと同一とみなし削除（Noneを返すことでフィルタリング）
    if feature == 'Pulse_Area_norm':
        return None
        
    # ChannelとFeatureを結合 
    if channel in ['Red', 'IR']:
        return f"{channel}_{feature}"
    elif channel == 'Red/IR_Ratio':
        # Ratioの場合はFeature名をそのまま使用（例: Red_IR_amp_norm_ratio）
        return feature
    return None

df_filtered_plot['Combined_Feature'] = df_filtered_plot.apply(format_feature_name, axis=1)
df_filtered_plot = df_filtered_plot.dropna(subset=['Combined_Feature'])

# 2.2. 色の定義
def get_feature_color(feature_name):
    """Red/IRに基づいて色を決定"""
    feature_name = feature_name.lower()
    if 'red' in feature_name:
        return 'red'
    elif 'ir' in feature_name:
        return 'purple'
    return 'black'

# 2.3. ヒートマップ描画用データフレームの準備
pair_order = [
    'Normal vs Ischaemia',
    'Normal vs Congestion',
    'Ischaemia vs Congestion'
]
depth_order = sorted(df_filtered_plot['Depth'].unique())

text_grid = {} # key: (Pair, Depth), value: [(text, color, auc_value), ...]
heatmap_values = pd.DataFrame(index=pair_order, columns=depth_order, dtype=float)

for index, row in df_filtered_plot.iterrows():
    pair = row['Pair']
    depth = row['Depth']
    feature_name = row['Combined_Feature']
    auc_value = row['AUC_bootstrap_mean']
    
    key = (pair, depth)
    
    # テキスト準備
    text = f"{feature_name}: {auc_value:.2f}"
    color = get_feature_color(feature_name)
    
    if key not in text_grid:
        text_grid[key] = []
    # (text, color, auc_value) のタプルでAUC値も格納
    text_grid[key].append((text, color, auc_value))
    
    # 背景色の基準（セル内の最大AUC）を更新
    if pair in pair_order and depth in depth_order:
        current_max = heatmap_values.loc[pair, depth]
        if pd.isna(current_max) or auc_value > current_max:
            heatmap_values.loc[pair, depth] = auc_value


# AUCが0.7未満（プロット対象外）のセルは、カラーマップの最小値（0.5）で埋める
MIN_AUC_FOR_CMAP = 0.5 
heatmap_values = heatmap_values.fillna(MIN_AUC_FOR_CMAP) 

# ========= 3. ヒートマップのプロット =========

fig, ax = plt.subplots(figsize=(13, 4)) 

# 3.1. 背景色
cmap = plt.cm.get_cmap('YlGnBu_r') 
norm = mcolors.Normalize(vmin=MIN_AUC_FOR_CMAP, vmax=1.0) 

cax = ax.imshow(heatmap_values.values, cmap=cmap, norm=norm, aspect='auto', interpolation='nearest')

# 3.2. 軸の設定
ax.set_xticks(np.arange(len(depth_order)))
ax.set_yticks(np.arange(len(pair_order)))
ax.set_xticklabels([f'{d} mm' for d in depth_order])
ax.set_yticklabels(pair_order)

ax.set_xlabel('Depth (mm)', fontsize=13)
ax.set_ylabel('Pairwise Comparison', fontsize=13)
ax.set_title(f'High AUC Features (Top 5)', fontsize=16, pad=20)

# 3.3. テキストの描画
for i, pair in enumerate(pair_order):
    for j, depth in enumerate(depth_order):
        key = (pair, depth)
        
        # AUC > 0.7のデータがあるセルのみにテキストを描画
        if key in text_grid and heatmap_values.loc[pair, depth] > 0.7:
            
            # ★★★ 修正箇所: AUCで降順ソートし、上位5つに制限 ★★★
            # text_grid[key] は [(text, color, auc_value), ...]
            sorted_entries = sorted(text_grid[key], key=lambda x: x[2], reverse=True)
            
            # 上位5つの (text, color) のみを取得
            top_5_entries = [(t, c) for t, c, a in sorted_entries[:5]] 
            
            n_entries = len(top_5_entries)
            
            # 垂直方向の中心から上下にずらして配置を調整
            # 5行まで対応するようにオフセットを調整
            y_offsets = np.linspace(-0.35, 0.35, n_entries) if n_entries > 0 else []
            
            for k, (text, color) in enumerate(top_5_entries):
                ax.text(j, i + y_offsets[k],
                        text, 
                        ha="center", 
                        va="center", 
                        color=color, 
                        fontsize=8, 
                        linespacing=1.2)

# 3.4. カラーバーの追加と修正
cbar_ticks = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
cbar = fig.colorbar(cax, ticks=cbar_ticks, label='AUC Value')

custom_labels = [f'{t:.1f}' for t in cbar_ticks]
custom_labels[0] = f'≤ {MIN_AUC_FOR_CMAP:.1f}'

cbar.ax.set_yticklabels(custom_labels) 

plt.tight_layout()
plt.show()